## Download graph 
#### This notebook downloads all nodes and edges we need and saves them in a csv file

We used data from: https://geoffboeing.com/publications/osmnx-complex-street-networks/

In [1]:
import osmnx as ox
import networkx as nx
import pandas as pd
import geopandas as gpd
import json

In [2]:
G = ox.graph_from_place("Rio de Janeiro, Rio de Janeiro, Brazil", network_type="drive", simplify=True)

In [3]:
try:
    G = ox.add_edge_speeds(G)
    G = ox.add_edge_travel_times(G)
except:
    print("Não tem esse dado aqui")
    pass

In [4]:
nodes = {}
node_list = []
old_node_id = {}
# Criando dicionário com os nós numerados de 0 a 1
for num, i in enumerate(G.nodes(data=True)):
    new_id = num+1
    # Definindo o novo id dos nós como chaves do dicionário
    nodes[new_id] = {'y':i[1]['y'], 'x':i[1]['x']}

    node_list.append([new_id ,i[1]['y'], i[1]['x'] ])
    old_node_id[i[0]]=new_id

nos = pd.DataFrame(node_list)
# nos.columns = ['id','long','lat']
nos.columns = ['id','y','x']
nos.to_csv('Nodes.csv', sep=';', index=False)
nos.head()

,id,y,x
0,1,-22.980949,-43.206859
1,2,-22.981045,-43.204756
2,3,-22.981160,-43.202639
3,4,-22.981267,-43.200507
4,5,-22.981470,-43.206925


In [5]:
# Organizando os id em um json
with open("node_coord.js", "w") as f:
    f.write("const node_coord = ")
    f.write(json.dumps(node_list))

In [6]:
edges_list = []
for num, j in enumerate(G.edges(data=True)):
    try: 
        name = j[2]['name']
    except:
        name = None
        
    edges_list.append([
                old_node_id[j[0]],
                old_node_id[j[1]],
                j[2]["travel_time"]
                # round(j[2]['length'],2)
                ])
                #round(j[2]['length'],2)])


edges = pd.DataFrame(edges_list)
edges.columns = ['id_from','id_to', 'travel_time']
edges.to_csv("Edges_travel_time.csv",sep=';', index=False, encoding='UTF-8')

In [7]:
edges_list = []
for num, j in enumerate(G.edges(data=True)):
    try: 
        name = j[2]['name']
    except:
        name = None
        
    edges_list.append([
                old_node_id[j[0]],
                old_node_id[j[1]],
                # j[2]["travel_time"]
                round(j[2]['length'],2)
                ])
                #round(j[2]['length'],2)])


edges = pd.DataFrame(edges_list)
edges.columns = ['id_from','id_to', 'travel_length']
edges.to_csv("Edges_travel_length.csv",sep=';', index=False, encoding='UTF-8')

In [8]:
nodes, edges = ox.graph_to_gdfs(G)
geometry_column=edges.reset_index()["geometry"]
from_column=edges.reset_index()["u"]
to_column=edges.reset_index()["v"]

for row in range(len(from_column)):
    from_column[row] = old_node_id[from_column[row]]
    to_column[row] = old_node_id[to_column[row]]

clean_edges = edges.reset_index()[["u","v", "geometry"]]
clean_edges["u"] = from_column
clean_edges["v"] = to_column

In [9]:
clean_edges.to_file("streets.geojson", 
    driver="GeoJSON",
    drop_id=True)

with open("streets.geojson","r") as f:
    content = f.read()

with open("streets.geojson","w") as f:
    f.write("var streets1 =")
    f.write(content)

In [10]:
streets1 = json.loads(content)

In [11]:
streets2 = {}
for num in range(len(streets1["features"])):
    element = streets1["features"][num]
    key = element["properties"]["u"]
    streets2[key] = []

for num in range(len(streets1["features"])):
    element = streets1["features"][num]
    key = element["properties"]["u"]

    dict_temp = {}
    dict_temp["type"] = "FeatureCollection"
    dict_temp["features"] = [element]

    streets2[key].append(dict_temp)

with open("streets2.js","w") as f:
    f.write("var streets2 =")
    f.write(str(streets2))


In [12]:
# import matplotlib.pyplot as plt

# Plot the streets
# fig, ax = ox.plot_graph(G)

# Retrieve nodes and edges
# nodes, edges = ox.graph_to_gdfs(G)
# nodes.head()

# # Plot the nodes
# fig, ax = plt.subplots(figsize=(12,8))
# nodes.plot(ax=ax, facecolor='black')